In [1]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('../../../final_project/DATA/KOSPI200_JONGMOK/A012450분봉데이터.csv', index_col=0)
df.head()

,date,time,open,high,low,close,volume,Accumulated sales volume,Accumulated number of contract
0,20230629,1530,125200,125200,125200,125200,45553,312742,377115
1,20230629,1520,125400,125500,125300,125300,2449,312742,333055
2,20230629,1519,125300,125400,125300,125400,2731,311538,331810
3,20230629,1518,125300,125400,125200,125400,1789,311233,329384
4,20230629,1517,125300,125300,125200,125300,1209,311109,327719


In [23]:
df.set_index(['date','time'])

open    high     low   close  volume   
date     time                                           
20230629 1530  125200  125200  125200  125200   45553  \
         1520  125400  125500  125300  125300    2449   
         1519  125300  125400  125300  125400    2731   
         1518  125300  125400  125200  125400    1789   
         1517  125300  125300  125200  125300    1209   
...               ...     ...     ...     ...     ...   
         905   126500  126700  126200  126200    5442   
         904   127100  127200  126400  126500    6644   
         903   127200  127600  127000  127100    4328   
         902   126700  127400  126100  127200    8421   
         901   127100  128200  126700  126700    5714   

               Accumulated sales volume  Accumulated number of contract  
date     time                                                            
20230629 1530                    312742                          377115  
         1520                    312742                          333055  
         1519                    311538                          331810  
         1518                    311233                          329384  
         1517                    311109                          327719  
...                                 ...                             ...  
         905                      15263                           14302  
         904                      12058                           12065  
         903                       7857                            9622  
         902                       6303                            6848  
         901                       1592                            3138  

[381 rows x 7 columns]

In [25]:
def normalize_data(dataset):
    cols = dataset.columns.tolist()
    col_name = [0]*len(cols)
    for i in range(len(cols)):
        col_name[i] = i
    dataset.columns = col_name
    dtypes = dataset.dtypes.tolist()
    minmax = list()
    for column in dataset:
        dataset = dataset.astype({column: 'float32'})
    for i in range(len(cols)):
        col_values = dataset[col_name[i]]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    for column in dataset:
        values = dataset[column].values
        for i in range(len(values)):
            values[i] = (values[i] - minmax[column][0]) / (minmax[column][1] - minmax[column][0])
        dataset[column] = values
    dataset[column] = values
    return dataset,minmax
    
dataset,minmax = normalize_data(df)
print(df.values)
values = dataset.values

[[20230629     1530   125200 ...    45553   312742   377115]
 [20230629     1520   125400 ...     2449   312742   333055]
 [20230629     1519   125300 ...     2731   311538   331810]
 ...
 [20230629      903   127200 ...     4328     7857     9622]
 [20230629      902   126700 ...     8421     6303     6848]
 [20230629      901   127100 ...     5714     1592     3138]]


C:\Users\green\AppData\Local\Temp\ipykernel_18768\309622529.py:19: RuntimeWarning: invalid value encountered in double_scalars
  values[i] = (values[i] - minmax[column][0]) / (minmax[column][1] - minmax[column][0])


In [26]:
def split_sequences(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
    
def data_setup(n_steps, n_seq,sequence):
    X, y = split_sequences(sequence, n_steps)
    n_features = X.shape[2]
    X = X.reshape((len(X),n_steps, n_features))
    new_y = []
    for term in y:
        new_term = term[-1]
        new_y.append(new_term)
    return X, np.array(new_y), n_features
    
n_steps = 10
n_seq = 10000
rel_test_len = 0.1
X,y,n_features = data_setup(n_steps,n_seq,values)
X = X[:-1]
y = y[1:]
X_train = X.reshape(X.shape[0], 28, 28, 1).astype('float32')
X_test = X.reshape(X.shape[0], 28, 28, 1).astype('float32')
X.shape

ValueError: cannot reshape array of size 33300 into shape (370,28,28,1)

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten,Conv1D, Conv2D,MaxPooling1D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
# from keras.layers.convolutional import Conv1D
# from keras.layers.convolutional import MaxPooling1D
model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(10,4)))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mse', optimizer='adam')

In [13]:
model.fit(X_train, y_train, epochs=70, batch_size=30)
pred_y = model.predict(X_test)

Epoch 1/70


ValueError: in user code:

    File "c:\Users\green\anaconda3\envs\pandas-dev\lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\green\anaconda3\envs\pandas-dev\lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\green\anaconda3\envs\pandas-dev\lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\green\anaconda3\envs\pandas-dev\lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\green\anaconda3\envs\pandas-dev\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\green\anaconda3\envs\pandas-dev\lib\site-packages\kera